In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#load the historical data
data = pd.read_csv(r'C:\Users\giras\Downloads\applstock.csv')

In [4]:
#feature scaling
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close_scaled'] = scaler.fit_transform(data['close'].values.reshape(-1, 1))

In [5]:
#Create sequences for input data 
lookback = 10
X, y = [], []

In [6]:
# Adjusting -3 to match the prediction horizon
for i in range(len(data) - lookback - 3):
    X.append(data['Close_scaled'].values[i:i + lookback])
    y.append(data['Close_scaled'].values[i + lookback:i + lookback + 3])  # Predict next 3 days

In [7]:
X, y = np.array(X), np.array(y)

In [8]:
#Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
#Reshape input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [10]:
#Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=3))  # Output layer for predicting next 3 days

C:\Users\giras\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
#compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [12]:
#train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 0.0387
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0020
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 8.2377e-04
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 7.9286e-04
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 7.2689e-04
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 7.7033e-04
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 9.0322e-04
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 7.1644e-04
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 9.4148e-04
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 7.9657e-04
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 7.4219e-04
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 8.3576e-04
Epoch 13/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 8.1422e-04
Epoch 14/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 7.3630e-04
Epoch 15/20
32/32 ━━━━━

In [13]:
#evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0025      
Mean Squared Error on Test Set: 0.004785815253853798


In [22]:
#make predictions on the data
new_data = [100.0, 105.0, 110.0, 115.0, 120.0]
new_data_scaled = scaler.transform(np.array(new_data).reshape(-1, 1))
new_data_reshaped = np.reshape(new_data_scaled, (1,5,1))
predictions_scaled = model.predict(new_data_reshaped)
predictions = scaler.inverse_transform(predictions_scaled.reshape(-1, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step


In [23]:
#Print predicted stock prices for the next 3 days
print('Predicted Stock Prices for the Next 3 Days:')
print(predictions)

Predicted Stock Prices for the Next 3 Days:
[[104.4811  ]
 [105.098   ]
 [104.412895]]
